In [1]:
#!python -m pip uninstall -y whisper
!pip install git+https://github.com/MiscellaneousStuff/whisper.git

  Cloning https://github.com/MiscellaneousStuff/whisper.git to /tmp/pip-req-build-yhky6pw3
  Running command git clone --filter=blob:none --quiet https://github.com/MiscellaneousStuff/whisper.git /tmp/pip-req-build-yhky6pw3
  Resolved https://github.com/MiscellaneousStuff/whisper.git to commit e87e7c46466505688119011f9190f7eb8c437b53
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB

In [2]:
import whisper
import torch
print(whisper.__file__)

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py


In [3]:
model_fp32 = whisper.load_model(
    name="base",
    device="cpu")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 87.5MiB/s]


In [5]:
quantized_model = torch.quantization.quantize_dynamic(
    model_fp32, {torch.nn.Linear}, dtype=torch.qint8
)

In [6]:
import os

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")/1e6
    print('Size (MB):', size)
    os.remove('temp.p')
    return size

print_size_of_model(model_fp32)
print_size_of_model(quantized_model)

Size (MB): 290.444061
Size (MB): 158.394589


158.394589

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# save/load using state dict
path = '/content/drive/My Drive/NLP/qwhisper.pth'
torch.save(quantized_model.state_dict(), path)

In [ ]:
# path = '/home/omerbeg/Documents/My Courses/NLP/Lectures/25-Speech Processing/quant/qwhisper.pth'
# device = torch.device('cpu')
# quantized_model = torch.load(path, map_location=device)

In [14]:
test_path = '/content/drive/My Drive/NLP/045FCN008.wav'
audio = whisper.load_audio(test_path)
audio = whisper.pad_or_trim(audio)

mel   = whisper.log_mel_spectrogram(audio).to(model_fp32.device)
options = whisper.DecodingOptions()

In [15]:
# regular
_, probs = model_fp32.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: hi


In [16]:
# quantized
_, probs = quantized_model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: hi


In [17]:
import time
def time_model_evaluation(model, mel, options):
    eval_start_time = time.time()
    # result = whisper.decode(model, mel, options)
    result = whisper.transcribe(model, test_path,  language="ur",)# , options)
    eval_end_time = time.time()
    eval_duration_time = eval_end_time - eval_start_time
    print(result["text"])
    print("Evaluate total time (seconds): {0:.1f}".format(eval_duration_time))

## Evaluate the original FP32 WHISPER model
time_model_evaluation(model_fp32, mel, options)

# Evaluate the INT8 WHISPER model after the dynamic quantization
time_model_evaluation(quantized_model, mel, options)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:76: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 اس پوست کو شیر کریں
Evaluate total time (seconds): 2.3


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:76: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 اس پوست کو شیر کریں
Evaluate total time (seconds): 2.1
